<a href="https://colab.research.google.com/github/MMBAYAR/GUZ-24_ISLE421/blob/main/Ula%C5%9Ft%C4%B1rma%26SetCovering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#  gurobipy yazılımını kur ve çalıştır
!pip install gurobipy
from gurobipy import*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 51.5 MB/s eta 0:00:00


In [2]:
#  Veri

#    Birim ulaştırma maliyetleri
C_u = [
      (10,    12,    11,    10**10,10**10,10**10,2,     1,     3     ),
      (10**10,10**10,10**10,4,     3,     5,     7,     11,    2     ),
      (3,     7,     2,     8,     1,     4,     10**10,10**10,10**10),
      (10,    11,    10**10,12,    13,    10**10,20,    13,    10**10)
      ]

#    Arsa satın alım bedelleri
F_a = [3000,4000,5000,7000,6000,8000,11000,12000,13000]

#    Şehirlerin talepleri
D_j = [10000,9000,12000,11000]

#    Birim inşaat maliyeti
v = 100

In [5]:
#   Bir model objesini m değişkenine sabitliyorum
m = Model()
#   Karar Değişkenleri
#    Arsa alım kararları
a = m.addVars(
              9,           #  len(F_a)
              vtype = 'b'
              )
#    Kurulum (inşaat) kararları
k = m.addVars(
              9,           #  len(F_a)
              vtype = 'i'
              )
#    Sevkiyat (ulaştırma) kararları
u = m.addVars(
              4,            #  len(D_j)  #  len(C_u)
              9,            #  len(F_a)  #  len( C_u[0] )
              vtype = 'c'
              )

#  Amaç Fonksiyonu
m.setObjective(
              quicksum( F_a[i]*a[i] for i in range(len(F_a)) )                                  #  Arsa alım maliyetleri
              + quicksum( v*k[i] for i in range(len(F_a)) )                                     #  İnşaat maliyetleri
              + quicksum( C_u[j][i]*u[j,i] for j in range(len(D_j)) for i in range(len(F_a)) )  #  Ulaştırma maliyetleri
              , GRB.MINIMIZE
              )

#  Kısıtlar
#    Mülkiyeti bende olmayan arsaya inşaat yapamam
for i in range(9):
  m.addConstr(
              k[i] <= (10**10)*a[i]
              )
#    Şehirlerin talepleri karşılansın
for j in range(4):
  m.addConstr(
              quicksum( u[j,i] for i in range(len(F_a)) ) >= D_j[j]
              )
#   Sevkiyatlar tesis kapasitelerini aşmasın
for i in range(9):
  m.addConstr(
              quicksum( u[j,i] for j in range(len(D_j)) ) <= 10*k[i]
              )

#  Çöz!
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 22 rows, 54 columns and 99 nonzeros
Model fingerprint: 0xee2545a1
Variable types: 36 continuous, 18 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  Objective range  [1e+00, 1e+10]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+03, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 9.000000e+13
Presolve removed 9 rows and 9 columns
Presolve time: 0.00s
Presolved: 13 rows, 45 columns, 81 nonzeros
Found heuristic solution: objective 9.000000e+13
Variable types: 36 continuous, 9 integer (0 binary)

Root relaxation: objective 5.700000e+05, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |

In [6]:
a

{0: <gurobi.Var C0 (value 1.099999999995e-07)>,
 1: <gurobi.Var C1 (value -0.0)>,
 2: <gurobi.Var C2 (value -0.0)>,
 3: <gurobi.Var C3 (value -0.0)>,
 4: <gurobi.Var C4 (value 1.199999999995e-07)>,
 5: <gurobi.Var C5 (value -0.0)>,
 6: <gurobi.Var C6 (value -0.0)>,
 7: <gurobi.Var C7 (value 9.99999999995e-08)>,
 8: <gurobi.Var C8 (value 8.99999999995e-08)>}